In [25]:
####################################################################################
# Import Packages
####################################################################################
# Localhost
# http://localhost:4040/ 

# Packages
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import os
import sys
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import rank
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.sql.types import StringType, ArrayType,StructType,StructField


spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [26]:
import pandas as pd
titanic = pd.read_csv("/Users/largo/Google Drive/DS/Python/5_Importing_Data_1/Data/titanic_sub.csv", sep=",", header=0)
titanic[['Sex', 'Ticket', "Cabin", "Embarked"]] = titanic[['Sex', 'Ticket', "Cabin", "Embarked"]].astype(str)
titanic = spark.createDataFrame(titanic)
titanic.show()

+-----------+--------+------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|  male|22.0|    1|    0|       A/5 21171|   7.25|  nan|       S|
|          2|       1|     1|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|female|26.0|    0|    0|STON/O2. 3101282|  7.925|  nan|       S|
|          4|       1|     1|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|  male|35.0|    0|    0|          373450|   8.05|  nan|       S|
|          6|       0|     3|  male| NaN|    0|    0|          330877| 8.4583|  nan|       Q|
|          7|       0|     1|  male|54.0|    0|    0|           17463|51.8625|  E46|       S|
|          8|       0|     3|  male| 2.0|    3|    1|       

In [27]:
titanic.schema

StructType(List(StructField(PassengerId,LongType,true),StructField(Survived,LongType,true),StructField(Pclass,LongType,true),StructField(Sex,StringType,true),StructField(Age,DoubleType,true),StructField(SibSp,LongType,true),StructField(Parch,LongType,true),StructField(Ticket,StringType,true),StructField(Fare,DoubleType,true),StructField(Cabin,StringType,true),StructField(Embarked,StringType,true)))

In [28]:
# Remap the values in "Pclass" to Proletarian/Ambitious/Burgouise OK
# Remap the values in "Sex" to 0/1 OK
# Take log of "Age" OK
# Clean the column "Fare" for inflation OK
# Extract the number from the column "Ticket"
# Fill in the missing values in the Cabin column
from pyspark.sql.types import StringType, BooleanType, DateType

def return_function(data, prefix, gender):
    df = data
    to_drop = ["PassengerId", "SibSp"]
    df = df.drop(*to_drop)
    df = df.withColumn("Class",  when(df.Pclass == 3, "Prol").when(df.Pclass == 2, "Ambitious").when(df.Pclass == 1, "Burgeous"))
    df = df.withColumn("Sex", when(df.Sex=="male", "MALE").when(df.Sex=="female", "FEMALE"))
    df = df.withColumn("Fare", df.Fare*100)
    df = df.withColumn("Parch_New", F.when(F.col("Parch")==0, F.array(F.lit('666'))))
    df = df.withColumn("Parch_New", F.when(F.col("Parch")==0, F.array(F.lit("666"), F.lit("777"))))


    # Write a function, add a column, if Parch_New=666 then "first" otherwise "second"

    # Dictionary Loop to recast DataTypes
    column_types = {
        "StringType()": ["Sex", "Ticket", "Cabin", "Embarked"],
        "IntegerType()": ["Pclass", "Age", "SibSp", "Parch"],
        "BooleanTyp0()": ["Survived"]
        }

    for column in df.columns:
        if column in column_types.items():
            df = df.withColumn("item", df.item.cast("key"))

    """
        for key, value in age_group.items():
        if df.Age in value:
            df = df[key] 
    """

    #df = df.withColumn("Survived", df.Survived.cast(BooleanType()))
    #df = df.withColumn("Sex", df.Sex.cast(StringType()))
    #df = df.withColumn("Age", df.Age.cast(IntegerType()))



    # Write a loop that recasts the column types
    string_typo = ["Sex", "Ticket", "Cabin", "Embarked"]
    if string_typo in df.columns:
        for i in string_typo:
            df = df.withColumn(i, df.i.cast("String"))    
    
 

    # This renames the columns
    for name in df.columns:
        if prefix not in df.columns:
            df = df.withColumnRenamed(name, name + '_' + prefix)
        else:
            raise ValueError('The chosen prefix already appers in the dataframe.')
    

    return df

new_df = return_function(titanic, "Y", "MALE")
new_df.show(truncate=False)





+----------+--------+------+-----+-------+----------------+-----------------+-------+----------+---------+-----------+
|Survived_Y|Pclass_Y|Sex_Y |Age_Y|Parch_Y|Ticket_Y        |Fare_Y           |Cabin_Y|Embarked_Y|Class_Y  |Parch_New_Y|
+----------+--------+------+-----+-------+----------------+-----------------+-------+----------+---------+-----------+
|0         |3       |MALE  |22.0 |0      |A/5 21171       |725.0            |nan    |S         |Prol     |[666, 777] |
|1         |1       |FEMALE|38.0 |0      |PC 17599        |7128.33          |C85    |C         |Burgeous |[666, 777] |
|1         |3       |FEMALE|26.0 |0      |STON/O2. 3101282|792.5            |nan    |S         |Prol     |[666, 777] |
|1         |1       |FEMALE|35.0 |0      |113803          |5310.0           |C123   |S         |Burgeous |[666, 777] |
|0         |3       |MALE  |35.0 |0      |373450          |805.0000000000001|nan    |S         |Prol     |[666, 777] |
|0         |3       |MALE  |NaN  |0      |330877

In [36]:
#######################################
# Removing characters from array
#######################################
new_df_2 = new_df.withColumn("REMOVED", F.split(new_df["Parch_New_Y"], " "))
new_df_2.show()
#new_df_2 = new_df_2.withColumnRenamed("REMOVED", F.regexp_replace(F.col("Parch_New_Y"), "[", ""))


+----------+--------+------+-----+-------+----------------+-----------------+-------+----------+---------+-----------+-------------------+
|Survived_Y|Pclass_Y| Sex_Y|Age_Y|Parch_Y|        Ticket_Y|           Fare_Y|Cabin_Y|Embarked_Y|  Class_Y|Parch_New_Y|            REMOVED|
+----------+--------+------+-----+-------+----------------+-----------------+-------+----------+---------+-----------+-------------------+
|         0|       3|  MALE| 22.0|      0|       A/5 21171|            725.0|    nan|         S|     Prol| [666, 777]|       [A/5, 21171]|
|         1|       1|FEMALE| 38.0|      0|        PC 17599|          7128.33|    C85|         C| Burgeous| [666, 777]|        [PC, 17599]|
|         1|       3|FEMALE| 26.0|      0|STON/O2. 3101282|            792.5|    nan|         S|     Prol| [666, 777]|[STON/O2., 3101282]|
|         1|       1|FEMALE| 35.0|      0|          113803|           5310.0|   C123|         S| Burgeous| [666, 777]|           [113803]|
|         0|       3|  MALE

In [12]:
##################################################
# Partitioning and selecting
##################################################
df =titanic
window_sep = Window.partitionBy("SEX").orderBy("Ticket")
new_df = df.withColumn("RANK", F.rank().over(window_sep))
new_df.filter(new_df["RANK"]==1).show()

+-----------+--------+------+------+----+-----+-----+------+-----+-----+--------+----+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|Ticket| Fare|Cabin|Embarked|RANK|
+-----------+--------+------+------+----+-----+-----+------+-----+-----+--------+----+
|        258|       1|     1|female|30.0|    0|    0|110152| 86.5|  B77|       S|   1|
|        505|       1|     1|female|16.0|    0|    0|110152| 86.5|  B79|       S|   1|
|        760|       1|     1|female|33.0|    0|    0|110152| 86.5|  B77|       S|   1|
|        263|       0|     1|  male|52.0|    1|    1|110413|79.65|  E67|       S|   1|
+-----------+--------+------+------+----+-----+-----+------+-----+-----+--------+----+



In [13]:
import pandas as pd
titanic = pd.read_csv("/Users/largo/Google Drive/DS/Python/5_Importing_Data_1/Data/titanic_sub.csv", sep=",", header=0)
titanic[['Sex', 'Ticket', "Cabin", "Embarked"]] = titanic[['Sex', 'Ticket', "Cabin", "Embarked"]].astype(str)
titanic = titanic.iloc[0:4, :]

def return_function(data):
    # This does some random stuff
    df = data

    # Create age buckets
    age_group = {
        "young": range(0,25),
        "middleaged": range(26,50),
        "old": range(51,110)
        }


    for key, value in age_group.items():
        if df.Age in value:
            df = df[key]
    
    return df

new_df = return_function(titanic)
print(new_df)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [14]:
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)]
schema = ["employee_name", "department", "salary"]
table = spark.createDataFrame(data=simpleData, schema=schema)

def recasting_function(data):
    df = data

    column_types = {
        "StringType()": ["employee_name", "department"],
        "IntegerType()": ["salary"]
        }

    for column in df.columns:
        if column in column_types.items():
            df = df.withColumn(item, df.item.cast(key))
    
    return df
        

result = recasting_function(table)

In [15]:
result.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



In [89]:
# This creates a sample dataframe
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)]
schema = ["employee_name", "department", "salary"]
table = spark.createDataFrame(data=simpleData, schema=schema)

# This is the function which is supposed to change datatypes en bulk
def recasting_function(data):
    column_types = {
        "string": ["employee_name", "department", "salary"]
        }
    for (k, v) in column_types.items():
        for c in v:
            if c in data.columns:
                data = data.withColumn(c, data[c].cast(k))    
    return data
        
# Here I apply it to my sample dataset
result = recasting_function(table)